In [1]:
!ls 

README.md                          env
analysis.ipynb                     meilleur_proposition_pour_oppo.sql
data_sources


In [3]:
pip install pandas

  Using cached pytz-2024.2-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 3.6 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.2/21.2 MB 7.1 MB/s eta 0:00:00a 0:00:01
Using cached pytz-2024.2-py2.py3-none-any.whl (508 kB)

[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [53]:
import pandas as pd
from typing import Dict
from dataclasses import dataclass

In [6]:
df_oportunity = pd.read_csv("data_sources/data_samples/opportunity_test.csv")

In [11]:
df_oportunity.head(3)

,Id,A_une_proposition_de_sa_banque__c,Age_emprunteur__c,Appel_Automatique__c,Avancement__c,BanquePrincipaleEmp__c,Connaissances_en_immobilier__c,CreatedDate,Deja_souscrit_credit_immo__c,DurSouhaitePret__c,...,TechMail_ContratDeTravailCoEmprunteur__c,TechMail_ContratDeTravailEmprunteur__c,TotalProposition__c,TotCharges__c,TotRev__c,Tx_Completude_Pieces__c,TxEndetApres__c,TypBien__c,TypProj__c,UsagBien__c
0,0065q00000AlAK0AAN,0,33.0,0,2.0,CIC,2.0,2023-10-20 04:44:34,1,20.0,...,CDI période d'essai terminée,CDI période d'essai terminée,1.0,1750.0,7731.67,0.0,41.75,Autres,Acquisition+travaux,Investissement locatif
1,0065q00000AlAK5AAN,0,30.0,0,4.0,BOURSORAMA BANQUE,1.0,2023-10-20 04:44:34,0,20.0,...,NaN,CDI période d'essai terminée,NaN,0.0,1695.00,0.0,33.86,Appartement,Acquisition,Résidence principale
2,0065q00000AlAKPAA3,0,50.0,1,4.0,BNP PARIBAS,2.0,2023-10-20 04:56:59,1,20.0,...,NaN,CDD,4.0,500.0,25500.00,0.0,12.00,Maison,Acquisition,Investissement locatif


In [10]:
df_oportunity_ = df_oportunity[['Age_emprunteur__c', 'BanquePrincipaleEmp__c', 
                                'Deja_souscrit_credit_immo__c', 
                                'TechMail_CategorieProfessionnelleEmpru__c',
                                'TechMail_ContratDeTravailEmprunteur__c'
                                ]]
df_oportunity_.head()

,Age_emprunteur__c,BanquePrincipaleEmp__c,Deja_souscrit_credit_immo__c,TechMail_CategorieProfessionnelleEmpru__c,TechMail_ContratDeTravailEmprunteur__c
0,33.0,CIC,1,Salarié du privé,CDI période d'essai terminée
1,30.0,BOURSORAMA BANQUE,0,Salarié du privé,CDI période d'essai terminée
2,50.0,BNP PARIBAS,1,Travailleur indépendant,CDD
3,22.0,BNP PARIBAS,0,Salarié du privé,CDI période d'essai terminée
4,42.0,BANQUE POPULAIRE,1,Salarié du privé,CDI période d'essai terminée


## Partie 1 : Analyse

### Question 1 : Déterminer le profil des personnes ayant soumis des opportunités (les demandeurs de prêt immobilier)

En considérant uniquement les champs suivant : Age_emprunteur__c', 'BanquePrincipaleEmp__c', 'Deja_souscrit_credit_immo__c', 'TechMail_CategorieProfessionnelleEmpru__c', 'TechMail_ContratDeTravailEmprunteur__c' , on peut déterminer le profil des demandeurs en calculant certains indicateurs. On pourra aussi effectuer des analyses croisées.

In [58]:

@dataclass
class AnalyseDeProfil:
    """Classe pour analyser le profil des emprunteurs"""
    
    def __init__(self, df: pd.DataFrame):
        self.df = df
        self.clean_data()
    
    def clean_data(self):
        """Nettoie les données"""

        '''
            age_min = -940.0 est une valeur aberrante. Cela montre qu'il y'a un probleme au niveau des données.
            On peut effectuer un nétoyage des données suivi d'une validation. Il en est de meme pour age_max = 144.
            Donc il faut éliminer les valeurs aberrante ou fixer un interval d'age valable et ainsi eliminer 
            l'ensemble des lignes que ne matche pas.
        '''
        
        # Suppression des valeurs aberrantes pour l'âge
        self.df = self.df[self.df['Age_emprunteur__c'].between(18, 100)]
    
    def age_emprunteur_statistics(self) -> Dict:
        """Calcule les statistiques sur l'âge"""
        age_emprunteur_stats = {
            'moyenne': self.df['Age_emprunteur__c'].mean(),
            'mediane': self.df['Age_emprunteur__c'].median(),
            'min': self.df['Age_emprunteur__c'].min(),
            'max': self.df['Age_emprunteur__c'].max(),
            
        }
        
        return age_emprunteur_stats
    
    def banque_principale_analysis(self) -> Dict:
        """Analyse la répartition des banques principales"""
        return {
            'repartition': self.df['BanquePrincipaleEmp__c'].value_counts().to_dict(),
            'pourcentage': (self.df['BanquePrincipaleEmp__c'].value_counts(normalize=True) * 100).to_dict(),
            'nombre_total': len(self.df['BanquePrincipaleEmp__c'].unique())
        }
    
    def credit_immo_precedent(self) -> Dict:
        """Analyse de l'expérience précédente en crédit immobilier"""
        return {
            'repartition': self.df['Deja_souscrit_credit_immo__c'].value_counts().to_dict(),
            'pourcentage': (self.df['Deja_souscrit_credit_immo__c'].value_counts(normalize=True) * 100).to_dict()
        }
    
    def categorie_professionnelle_analysis(self) -> Dict:
        """Analyse sur les catégories professionnelles"""
        return {
            'repartition': self.df['TechMail_CategorieProfessionnelleEmpru__c'].value_counts().to_dict(),
            'pourcentage': (self.df['TechMail_CategorieProfessionnelleEmpru__c'].value_counts(normalize=True) * 100).to_dict(),
            'top_5': self.df['TechMail_CategorieProfessionnelleEmpru__c'].value_counts().head().to_dict()
        }
    
    def type_contract_travail_analysis(self) -> Dict:
        """Analyse les types de contrats de travail"""
        return {
            'repartition': self.df['TechMail_ContratDeTravailEmprunteur__c'].value_counts().to_dict(),
            'pourcentage': (self.df['TechMail_ContratDeTravailEmprunteur__c'].value_counts(normalize=True) * 100).to_dict()
        }
    
    
    
    def generer_rapport(self) -> Dict:
        """Génère un rapport complet d'analyse"""
        return {
            'statistiques_age': self.age_emprunteur_statistics(),
            'analyse_banques': self.banque_principale_analysis(),
            'analyse_experience_credit': self.credit_immo_precedent(),
            'analyse_categories_pro': self.categorie_professionnelle_analysis(),
            'analyse_contrats': self.type_contract_travail_analysis(),
        }

# Exemple d'utilisation
def main():
    # Chargement des données (à adapter selon votre source)
    
    df = pd.read_csv("data_sources/data_samples/opportunity_test.csv")
    df_oportunity_ = df[['Age_emprunteur__c', 'BanquePrincipaleEmp__c', 
                                'Deja_souscrit_credit_immo__c', 
                                'TechMail_CategorieProfessionnelleEmpru__c',
                                'TechMail_ContratDeTravailEmprunteur__c'
                                ]]
    
    # Création de l'analyseur
    analyzer = AnalyseDeProfil(df_oportunity_)
    
    # Génération du rapport
    rapport = analyzer.generer_rapport()
    
    # Affichage des résultats principaux
    print("\n=== Profil Type des Emprunteurs ===\n")
    
    # Age
    print("1. Profil d'âge:")
    print(f"- Âge moyen: {rapport['statistiques_age']['moyenne']:.1f} ans")
    print(f"- Âge médian: {rapport['statistiques_age']['mediane']:.1f} ans")
    
    # Catégorie professionnelle
    print("\n2. Top 5 des catégories professionnelles:")
    for cat, count in rapport['analyse_categories_pro']['top_5'].items():
        pct = rapport['analyse_categories_pro']['pourcentage'][cat]
        print(f"- {cat}: {count} personnes ({pct:.1f}%)")
    
    # Expérience crédit
    print("\n3. Expérience en crédit immobilier:")
    for exp, pct in rapport['analyse_experience_credit']['pourcentage'].items():
        print(f"- {exp}: {pct:.1f}%")
    
    # Type de contrat
    print("\n4. Types de contrat de travail:")
    for contract, pct in rapport['analyse_contrats']['pourcentage'].items():
        print(f"- {contract}: {pct:.1f}%")

if __name__ == "__main__":
    main()


=== Profil Type des Emprunteurs ===

1. Profil d'âge:
- Âge moyen: 37.3 ans
- Âge médian: 36.0 ans

2. Top 5 des catégories professionnelles:
- Salarié du privé: 125596 personnes (72.2%)
- Salarié du public: 27028 personnes (15.5%)
- Dirigeant de société: 4828 personnes (2.8%)
- Retraité: 4155 personnes (2.4%)
- Travailleur indépendant: 2801 personnes (1.6%)

3. Expérience en crédit immobilier:
- 0: 59.2%
- 1: 40.8%

4. Types de contrat de travail:
- CDI période d'essai terminée: 79.1%
- Fonctionnaire ou assimilé: 14.3%
- Éducation nationale: 2.3%
- CDD: 1.9%
- CDI période d'essai non terminée: 1.4%
- Intérimaire: 0.6%
- Autres: 0.3%
- Multi-employeurs: 0.0%
- Intermittent du spectacle: 0.0%


### Question 2 :